### Multiple Linear Regression w/ Ridge & LASSO + unsuccessful attempt at log transform

#### DO NOT RUN ALL, WILL GET ERRORS
____

Import libraries

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

%matplotlib inline

In [2]:
train = pd.read_csv('data/train_clean.csv')
test = pd.read_csv('data/test_clean.csv')

In [3]:
train.head()

,Unnamed: 0,Id,parcel_id,ms_subclass,zoning,lot_frontage,lot_sq_ft,paved_street,alley,lot_shape,...,basement_unfinished,finished_basement_sqft,newly_renovated,functionality_salvage_only,functionality_severely_damaged,functionality_typical,building_type_single_family,building_type_townhouse_end_unit,building_type_townhouse_inside_unit,building_type_two_family_conversion
0,0,109,533352170,60,RL,0.0,13517,1,None,IR1,...,0,533.0,0,0,0,1,1,0,0,0
1,1,544,531379050,60,RL,43.0,11492,1,None,IR1,...,0,637.0,0,0,0,1,1,0,0,0
2,2,153,535304180,20,RL,68.0,7922,1,None,Reg,...,0,731.0,0,0,0,1,1,0,0,0
3,3,318,916386060,60,RL,73.0,9802,1,None,Reg,...,1,0.0,0,0,0,1,1,0,0,0
4,4,255,906425045,50,RL,82.0,14235,1,None,IR1,...,1,0.0,0,0,0,1,1,0,0,0


In [4]:
train.shape

(2051, 92)

In [5]:
train.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
train.shape

(2051, 91)

In [7]:
train.columns

Index(['Id', 'parcel_id', 'ms_subclass', 'zoning', 'lot_frontage', 'lot_sq_ft',
       'paved_street', 'alley', 'lot_shape', 'flatness', 'utilities',
       'lot_config', 'land_slope', 'neighborhood', 'proximity_1',
       'proximity_2', 'style', 'quality_rating', 'condition_rating',
       'year_built', 'year_renovation', 'roof_style', 'roof_material',
       'exterior_1', 'exterior_2', 'masonry_type', 'masonry_area',
       'exterior_quality', 'exterior_quality.1', 'foundation_type',
       'basement_height', 'basement_condition', 'basement_exposure',
       'basement_finished_condition', 'basement_fin_sqft',
       'basement_2_condition', 'basement_2_sqft', 'unfinished_basement_sqft',
       'total_basement_sqft', 'heating_type', 'heating_quality',
       'has_central_air', 'electrical', 'first_fl_sqft', 'second_fl_sqft',
       'low_quality_sqft', 'living_area_sqft', 'basement_full_bath',
       'basement_half_bath', 'full_bath', 'half_bath', 'bedrooms', 'kitchens',
       'kitchen

These are columns I chose based on correlations in EDA notebook.

In [8]:
good_cols = ['fireplaces', 'has_central_air','bedrooms', 'quality_rating',
                   'kitchens', 'lot_sq_ft', 'living_area_sqft','garage_size',
                   'baths', 'finished_basement_sqft', 'year_built',
                   'has_paved_driveway', 'functionality_typical', 'building_type_single_family', 
                   'building_type_townhouse_end_unit','building_type_townhouse_inside_unit',
                   'building_type_two_family_conversion', 'open_porch_sqft','enclosed_porch_sqft', 
                   'seasonal_porch_sqft', 'screen_porch_sqft', 'SalePrice']

In [9]:
features = features = [col for col in good_cols if col != 'SalePrice']

In [10]:
X = train[features]
y = np.log(train['SalePrice'])
X.head()

KeyError: 'SalePrice'

In [ ]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, random_state=42)

In [ ]:
ss = StandardScaler()

In [ ]:
ss.fit(X_train)

In [ ]:
X_train_sc = ss.transform(X_train)
X_holdout_sc = ss.transform(X_holdout)

In [ ]:
lr = LinearRegression()

In [ ]:
cross_val_score(lr, X_train_sc, y_train).mean()

In [ ]:
lasso = LassoCV(cv=5)

In [ ]:
cross_val_score(lasso, X_train_sc, y_train).mean()

In [ ]:
ridge = RidgeCV(cv=5)

In [ ]:
cross_val_score(ridge, X_train_sc, y_train).mean()

In [ ]:
ridge.fit(X_train_sc, y_train)

In [ ]:
#training score 
ridge.score(X_train_sc, y_train)

In [ ]:
#test(holdout) score
ridge.score(X_holdout_sc, y_holdout)

In [ ]:
X_holdout.head()

In [ ]:
y_train_pred = np.exp(ridge.predict(X_train_sc))
y_holdout_pred= np.exp(ridge.predict(X_holdout_sc))
# X_holdout['log_pred'] = np.exp(ridge.predict(X_holdout_sc))
#X_test['log_pred'] = np.exp(ridge.predict(X_test_sc))

In [ ]:
import math

In [ ]:
# X_holdout['yhat'] = X_holdout['log_pred'].apply(math.exp)

In [ ]:
X_test['yhat'] = X_test['log_pred'].apply(math.exp)

In [ ]:
yhat = X_holdout['yhat']

In [ ]:
y_holdout_exp = np.exp(y_holdout)
#actual

In [ ]:
y_hat

In [ ]:
X_holdout['yhat']

In [ ]:
residuals = y_holdout_exp - yhat
rss = (residuals**2).sum()

In [ ]:
RMSE = np.sqrt(rss / len(yhat))

In [ ]:
RMSE

In [ ]:
rmse(np.exp(y_holdout),y_holdout_pred)

In [ ]:
new_submission_df = pd.DataFrame({
    'Id': test['Id'],
    'SalePrice': y_hat   
})
new_submission_df.head()

In [ ]:
plt.scatter(pred, residuals)
plt.title("Difference in Predicted and Actual Price")
plt.xlabel("Predictions")
plt.ylabel("Residuals")

In [ ]:
test.head()

In [ ]:
test.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
X_test = test[features]
X_test.head()

In [ ]:
X_test_sc = ss.transform(X_test)

In [ ]:
y_hat = ridge.predict(X_test_sc)

In [ ]:
submission_df = pd.DataFrame({
    'Id': test['Id'],
    'SalePrice': y_hat   
})
submission_df.head()

In [ ]:
submission_df.to_csv('submission4.csv', index=False)

In [ ]:
rss = (residuals **2).sum()
rss

In [ ]:
len(pred)

In [ ]:
np.sqrt(rss / len(pred))

In [ ]:
r2_score(y_holdout, pred)

Since my residual line was curved, I am going to check the distribution and try something else.

In [ ]:
residuals.hist();
#Also the distribution of residuals is positively skewed so there is room to improve.

In [ ]:
y.hist();

Since both my residuals and y(target) are right skewed, I will try ..

In [ ]:
y.map(np.log).hist();
# A normally distributed target! 

In [ ]:
y_holdout_pt = y_holdout.map(np.log)
y_train_pt = y_train.map(np.log)

In [ ]:
cross_val_score(lr, X_train_sc, y_train_pt).mean()

In [ ]:
cross_val_score(lasso, X_train_sc, y_train_pt).mean()

In [ ]:
cross_val_score(ridge, X_train_sc, y_train_pt).mean()

In [ ]:
ridge.fit(X_train_sc, y_train_pt)

In [ ]:
#training score 
ridge.score(X_train_sc, y_train_pt)

In [ ]:
#holdout score
ridge.score(X_holdout_sc, y_holdout_pt)

In [ ]:
pred_transformed = np.exp(pred)

In [ ]:
pred = ridge.predict(X_holdout_sc)

residuals_pt = y_holdout_pt - pred

plt.scatter(pred, residuals_pt)

In [ ]:
rss = (residuals_pt**2).sum()
rss

In [ ]:
len(pred_transformed)

In [ ]:
np.sqrt(rss / len(pred_transformed))

In [ ]:
y_hat